In [16]:
from dotenv import load_dotenv
load_dotenv()
import os
from groq import Groq
from pinecone import Pinecone, ServerlessSpec


c:\Users\HP\miniconda3\envs\rag\lib\site-packages\pinecone\data\index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [35]:
pc=Pinecone(api_key=os.getenv("PINECONE_API_KEY"))
pc.delete_index("rag")
pc.create_index(
    name="rag",dimension=1024,metric="cosine",spec=ServerlessSpec(cloud="aws",region="us-east-1")
)

In [8]:
import json
data=json.load(open("reviews.json"))
data['reviews']

[{'professor': 'Dr. John Smith',
  'subject': 'Mathematics',
  'stars': 5,
  'review': 'Dr. Smith is an excellent professor. His lectures are clear, and he always makes time for students.'},
 {'professor': 'Prof. Lisa Turner',
  'subject': 'Physics',
  'stars': 4,
  'review': 'Prof. Turner is knowledgeable and engaging, but her exams are quite challenging.'},
 {'professor': 'Dr. Michael Brown',
  'subject': 'Chemistry',
  'stars': 3,
  'review': "Dr. Brown's lectures can be a bit dry, but the lab sessions are well-organized."},
 {'professor': 'Prof. Emily Davis',
  'subject': 'Biology',
  'stars': 5,
  'review': 'Prof. Davis is passionate about her subject and encourages students to think critically.'},
 {'professor': 'Dr. Robert Wilson',
  'subject': 'Computer Science',
  'stars': 2,
  'review': "Dr. Wilson's lectures are difficult to follow, and he rarely responds to emails."},
 {'professor': 'Prof. Sarah Johnson',
  'subject': 'History',
  'stars': 4,
  'review': 'Prof. Johnson is w

In [59]:
processed_data= []
client= Groq()
for review in data['reviews']:
    response=client.embeddings.create(
        input=review['review'],
        model="distil-whisper-large-v3-en",
    )
    embedding=response.data[0].embedding
    processed_data.append({
        "values":embedding,
        "id":review["professor"],
        "metadata":{
            "review":review["review"],
            "subject":review["subject"],
            "stars":review["stars"]

        }
    })


BadRequestError: Error code: 400 - {'error': {'message': 'The model `distil-whisper-large-v3-en` does not support embeddings', 'type': 'invalid_request_error'}}

In [37]:
processed_data[0]

IndexError: list index out of range

In [38]:
index=pc.Index('rag')
index.upsert(
    vectors=processed_data,
    namespace="ns1"

)

PineconeApiException: (400)
Reason: Bad Request
HTTP response headers: HTTPHeaderDict({'Date': 'Mon, 09 Sep 2024 17:18:15 GMT', 'Content-Type': 'application/json', 'Content-Length': '74', 'Connection': 'keep-alive', 'x-pinecone-request-latency-ms': '45', 'x-pinecone-request-id': '4524132751656479646', 'x-envoy-upstream-service-time': '46', 'server': 'envoy'})
HTTP response body: {"code":3,"message":"No vectors provided for upsert request","details":[]}


In [18]:
index.describe_index_stats()

{'dimension': 1024,
 'index_fullness': 0.0,
 'namespaces': {},
 'total_vector_count': 0}

In [9]:
import sys
print(sys.path)

['c:\\Users\\HP\\miniconda3\\envs\\rag\\python310.zip', 'c:\\Users\\HP\\miniconda3\\envs\\rag\\DLLs', 'c:\\Users\\HP\\miniconda3\\envs\\rag\\lib', 'c:\\Users\\HP\\miniconda3\\envs\\rag', '', 'c:\\Users\\HP\\miniconda3\\envs\\rag\\lib\\site-packages', 'c:\\Users\\HP\\miniconda3\\envs\\rag\\lib\\site-packages\\win32', 'c:\\Users\\HP\\miniconda3\\envs\\rag\\lib\\site-packages\\win32\\lib', 'c:\\Users\\HP\\miniconda3\\envs\\rag\\lib\\site-packages\\Pythonwin']
